# Finite volume solver for 1D Burgers' eq.

## Prerequisites (libraries and helper functions)
We start loading several libraries:

In [ ]:
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry
ngsglobals.msg_level = 1
from ngsolve.meshes import *
from draw import *

We may need function for absolute value and maxima computations on `CoefficientFunctions` which we compose with the `IfPos` function:

In [ ]:
def Abs(u):
    return IfPos(u,u,-u)
def Max(u,v):
    return IfPos(u-v,u,v)
def Min(u,v):
    return IfPos(u-v,v,u)

## A simple finite volume solver

We can now define a very simple finite volume solver based on:
 * a flux function $F$,
 * a numerical flux function $\hat{f}_n$,
 * initial values $u_0$,
 * boundary data (if needed),
 * a mesh and
 * a time step for an explicit Euler discretization.
 
Let $u_T$ be the constant corresponding to one element. Then the scheme reads as

$$
u_T^{n+1} = u_{T}^n - \frac{\Delta t}{|T|} \sum_{K \in \partial T} 
\hat{f}_{K,n}(u_T,u_{T'},n_K) |K|
$$
    

This, together with some plotting at $t = 0,0.1,..,0.5$ is done in the following function:

In [ ]:
def Solve(F, fhatn, u0, ubnd, mesh, dt):
    V = L2(mesh,order=0)
    gfu=GridFunction(V)
    u,v = V.TnT()
    # definition of the "bilinear" form
    # Note: It -- despite its name -- does not need to be linear in the first argument!
    a = BilinearForm (V, nonassemble=True)
    #a += -F(u)*grad(v) * dx #<- only for DG
    a += fhatn(F,u,u.Other(ubnd),specialcf.normal(mesh.dim)) * v * dx(element_boundary=True)
    t=0

    gfu.Set(u0)
    Draw(gfu,mesh,"u")
    Ts = [0.1,0.2,0.4]
    for T in Ts:
        while t < T-dt/2:
            gfu.vec.data -= dt * V.InvM() @ a.mat * gfu.vec
            t += dt
            Redraw()
        print("t = ", t, "total concentration: ", abs(Integrate(gfu,mesh,order=0)))
        Draw(gfu,mesh,"u")
    return gfu

## The mesh
For the mesh we take a N(xN) mesh. The time step is manually adjusted to provide stability in the sense of a CFL condition:

In [ ]:
N=100
mesh = Make1DMesh(n=N) #,periodic=True)
CFL=1
dt=CFL/N

The domain is $\Omega = (0,1)$ with the following boundary conditions:

In [ ]:
ubnd_dir = {"right" : 0.25, "left" : 0.75}

In [ ]:
ubnd = CoefficientFunction([ubnd_dir[key] for key in mesh.GetBoundaries()])

## The problem
As the model problem here we consider the LWR traffic model:
$$
F(u(x)) = u (1-u)
$$

In [ ]:
def F(u):
    return u * (1-u)

As initial values we prescribe $u_0(x(,y)) = \sin(2\pi x)$:

In [ ]:
u0 = IfPos(x-0.5,0.25,0.75)

## numerical examples

### A discretization with a Lax-Friedrichs flux

In [ ]:
def fhatn_central(F,u1,u2,n):
    return 0.5*F(u1)*n+0.5*F(u2)*n # central flux
def fhatn_LF(F,u1,u2,n): 
    #TODO:
    return None
gfu = Solve(F,fhatn_LF, u0, ubnd, mesh, dt)

In [ ]:
#compute error
sol = None #TODO!
print(Integrate(Abs(sol-gfu),mesh))

In [ ]:
def fhatn_EO(F,u1,u2,n): 
    # TODO
    return None
gfu = Solve(F,fhatn_EO, u0, ubnd, mesh, dt)

In [ ]:
print(Integrate(Abs(sol-gfu),mesh))